### SENSITIVITY CODE FOR AHP AND CONCORDANCE

In [1]:
import pandas as pd
import xlsxwriter
import numpy as np
from scipy.stats import gmean
from sympy import Point, Line, Segment

In [2]:
Weightages = pd.read_excel("Weightages.xlsx")
Weightages_lst = list(Weightages[0])
#print(Weightages_lst)

In [3]:
user_sensitivity_val = int(input("Enter the % of increment/decrement want to add for each value one at a time: "))
user_val = user_sensitivity_val/100

Enter the % of increment/decrement want to add for each value one at a time:  5


In [4]:
writer = pd.ExcelWriter('Sensitivity Weights for AHP and Concordance.xlsx', engine='xlsxwriter')
j = 0

for i in range(0,len(Weightages_lst)):
    Weightages_1 = Weightages_lst[:]
    sens_val_inc_dec = Weightages_1[i] + (Weightages_1[i] * user_val)
    Weightages_1[i] = sens_val_inc_dec
    globals()["sens_wts" + str(i)] = Weightages_1
    weights_df = globals()["sens_wts" + str(i)]
    weights_df = pd.DataFrame(weights_df)
    
    j += 1
    weights_df.to_excel(writer, sheet_name = "sens_wts" + str(j))

writer.save()    

#### CONCORDANCE CODE

In [5]:
FieldData = pd.read_excel("Field Data Concordance.xlsx")
FieldData = FieldData.set_index("STRETCH NO.")
#print(FieldData)

In [6]:
concordance = pd.ExcelWriter('Concordance rankings.xlsx', engine='xlsxwriter')
j = 1
for i in range(0,len(Weightages_lst)):
    sens_wts = pd.read_excel("Sensitivity Weights for AHP and Concordance.xlsx", sheet_name ="sens_wts" + str(j))
    sens_wts = list(sens_wts.iloc[:,-1])
    j += 1
    
    stretch_comp = []
    for k in (range(len(FieldData))):
        val1 = FieldData.iloc[k,:]
        for l in range(len(FieldData)):
            val2 = FieldData.iloc[l,:]

            for l in range(len(val1)):
                if val1[l] > val2[l]:
                    comp_vals = sens_wts[l]
                    stretch_comp.append(comp_vals)
                else:
                    #print(0)
                    stretch_comp.append(0)

    #print(stretch_comp)
    n = len(stretch_comp)// len(FieldData.columns)
    D1_array = np.array(stretch_comp)
    D2_array = np.reshape(D1_array, (n, len(FieldData.columns)))
    #print(D2_array)
    
    stretch_Df = pd.DataFrame(D2_array, columns = FieldData.columns)
    
    row_diff = ['m' + str(i) + "-" + "m" + str(j) for i in range(1, len(FieldData)+1) for j in range(1, len(FieldData)+1)]
    stretch_Df["mi-mj"] = row_diff
    stretch_Df = stretch_Df.set_index("mi-mj")
    
    sDF_rowWise_Sum = stretch_Df.loc[:].sum(axis = 1)
    stretch_Df["SUM"] = sDF_rowWise_Sum
    #print(stretch_Df)
    
    SUM_vals = np.array(stretch_Df["SUM"].tolist())
    SUM_vals_2D = np.reshape(SUM_vals, (len(FieldData), len(FieldData)))
    
    Ranking = pd.DataFrame(SUM_vals_2D)
    Ranking["STRETCH NO."] = FieldData.index
    Ranking = Ranking.set_index("STRETCH NO.")
    
    Ranking_rowWise_Sum = Ranking.loc[:].sum(axis = 1)
    Ranking["SUM"] = Ranking_rowWise_Sum
    #print(Ranking)
    
    Concordance_Ranking = pd.DataFrame(list(Ranking["SUM"]), columns = ["SUM"])
    Concordance_Ranking["STRETCH NO."] = Ranking.index
    Concordance_Ranking = Concordance_Ranking.set_index("STRETCH NO.")
    Concordance_Ranking["Rank"] = Concordance_Ranking["SUM"].rank(ascending = False, method = 'max')
    Concordance_Ranking.to_excel(concordance, sheet_name = "conc_rank" + str(i+1) + "_" +str(user_sensitivity_val))
    #print(str(i+1), Concordance_Ranking)
concordance.save()

#### AHP CODE

In [7]:
C_check_matrix_WSV = pd.read_excel("Consistency Check Matrix.xlsx")
C_check_matrix_WSV = C_check_matrix_WSV.set_index("CRITERIA")
Len_Matrix = len(C_check_matrix_WSV)
#print(Len_Matrix)
#print(C_check_matrix_WSV)

In [8]:
Normalized_Data = pd.read_excel("Check Normalized Data.xlsx")
Normalized_Data = Normalized_Data.set_index("STRETCH NO.")
Normalized_Data_Matrix = np.array(Normalized_Data)
#print(Normalized_Data_Matrix)

In [9]:
AHP = pd.ExcelWriter('AHP rankings.xlsx', engine='xlsxwriter')
j = 1
for i in range(0,len(Weightages_lst)):
    sens_wts = pd.read_excel("Sensitivity Weights for AHP and Concordance.xlsx", sheet_name ="sens_wts" + str(j))
    sens_wts = list(sens_wts.iloc[:,-1])
    j += 1
    
    C_check_matrix_WSV["Criteria Weights"] = list(sens_wts)
    #print(C_check_matrix_WSV)
    
    C_check_matrix_WSV['WSV / CW'] = C_check_matrix_WSV['WSV'] / C_check_matrix_WSV['Criteria Weights'] 
    λmax = gmean(C_check_matrix_WSV["WSV / CW"])
    
    Consistency_index = (λmax - Len_Matrix) / (Len_Matrix - 1)
    
    n = Len_Matrix
    if n > 0 and n < 51:
        Random_Index_vals={1:0, 2:0, 3:0.58, 4:0.90, 5:1.12, 6:1.24, 7:1.32, 8:1.41, 9:1.45, 10:1.49, 11:1.51, 12:1.48, 13:1.56, 14:1.57, 15:1.59,
                           16:1.6, 17:1.61, 18:1.61, 19:1.62, 20:1.63, 21:1.63, 22:1.64, 23:1.65, 24:1.65, 25:1.66, 26:1.66, 27:1.66, 28:1.67, 29:1.67, 30:1.67,
                           31:1.67, 32:1.68, 33:1.68, 34:1.68, 35:1.68, 36:1.69, 37:1.69, 38:1.69, 39:1.69, 40: 1.69, 41:1.70, 42:1.70, 43:1.70, 44:1.70, 45:1.70,
                           46:1.70, 47:1.70, 48:1.70, 49:1.71, 50:1.71}

        Random_Index = Random_Index_vals.get(n)
    else:
        Random_Index = float(input("Enter the Random Index according to no. of criterias: "))


    Consistency_Ratio = Consistency_index / Random_Index

    if Consistency_Ratio < 0.10:
        Consistency_Ratio = np.dot(Normalized_Data_Matrix, sens_wts)
        AHP_Ranking = pd.DataFrame()
        AHP_Ranking["Priority Index"] = Consistency_Ratio
        AHP_Ranking["STRETCHES"] = Normalized_Data.index
        AHP_Ranking = AHP_Ranking.set_index("STRETCHES")
        AHP_Ranking["Rank"] = AHP_Ranking["Priority Index"].rank(ascending = False, method = 'max')
        AHP_Ranking.to_excel(AHP, sheet_name = "ahp_rank" + str(i+1) + "_" +str(user_sensitivity_val))
        #print(str(i+1), AHP_Ranking)
    else:
        print('The provided metrics are "Inconsistent".')
        
AHP.save()

### SENSITIVITY CODE FOR TOPSIS

In [10]:
Fuzzy_wts = pd.read_excel("Fuzzy wts.xlsx")
Fuzzy_wts = Fuzzy_wts.set_index("Criteria")
#print(len(Fuzzy_wts))
Fuzzy_wts_list = list(Fuzzy_wts[2])
#print(Fuzzy_wts_list)

In [11]:
writer1 = pd.ExcelWriter('Sensitivity Weights for TOPSIS.xlsx', engine='xlsxwriter')
j = 0

for i in range(0,len(Fuzzy_wts)):
    Fuzzy_wts_list_1 = Fuzzy_wts_list[:]
    Fuzzy_wts_1 = Fuzzy_wts[:] 
    sens_val_inc_dec = Fuzzy_wts_list_1[i] + (Fuzzy_wts_list_1[i] * user_val)
    Fuzzy_wts_list_1[i] = sens_val_inc_dec
    globals()["sens_wts" + str(i)] = Fuzzy_wts_list_1
    weights_df = globals()["sens_wts" + str(i)]
    
    Fuzzy_wts_1[2] = weights_df
    weights_df_comp = pd.DataFrame(Fuzzy_wts_1)
    #print(weights_df_comp)
    
    j += 1
    weights_df_comp.to_excel(writer1, sheet_name = "sens_wts_TOPSIS" + str(j))

writer1.save()   

#### TOPSIS CODE

In [12]:
Normalized_data_topsis = pd.read_excel("Normalized Data Sheet.xlsx")
Normalized_data_topsis = Normalized_data_topsis.set_index("STRETCH NO.")
#print(Normalized_data_topsis)

In [13]:
Rating_Matrix = []
for i in range(0, len(Normalized_data_topsis.columns)):
    for j in range(0, len(Normalized_data_topsis)):
        if Normalized_data_topsis.iloc[j,i] >= 0 and Normalized_data_topsis.iloc[j,i] <= 10:
            Rating_Matrix.append(1)
        elif Normalized_data_topsis.iloc[j,i] >= 11 and Normalized_data_topsis.iloc[j,i] <= 20:
            Rating_Matrix.append(2)
        elif Normalized_data_topsis.iloc[j,i] >= 21 and Normalized_data_topsis.iloc[j,i] <= 30:
            Rating_Matrix.append(3)
        elif Normalized_data_topsis.iloc[j,i] >= 31 and Normalized_data_topsis.iloc[j,i] <= 40:
            Rating_Matrix.append(4)
        elif Normalized_data_topsis.iloc[j,i] >= 41 and Normalized_data_topsis.iloc[j,i] <= 50:
            Rating_Matrix.append(5)
        elif Normalized_data_topsis.iloc[j,i] >= 51 and Normalized_data_topsis.iloc[j,i] <= 60:
            Rating_Matrix.append(6)
        elif Normalized_data_topsis.iloc[j,i] >= 61 and Normalized_data_topsis.iloc[j,i] <= 70:
            Rating_Matrix.append(7)
        elif Normalized_data_topsis.iloc[j,i] >= 71 and Normalized_data_topsis.iloc[j,i] <= 80:
            Rating_Matrix.append(8)
        elif Normalized_data_topsis.iloc[j,i] >= 81 and Normalized_data_topsis.iloc[j,i] <= 90:
            Rating_Matrix.append(9)
        elif Normalized_data_topsis.iloc[j,i] >= 91 and Normalized_data_topsis.iloc[j,i] <= 100:
            Rating_Matrix.append(10)

cols = len(Normalized_data_topsis)
rows = len(Normalized_data_topsis.columns)
Rating_Matrix = np.array(Rating_Matrix)
Rating_Matrix = Rating_Matrix.reshape(rows,cols)
Rating_Matrix = Rating_Matrix.transpose()
#print(Rating_Matrix)

In [14]:
Topsis = pd.ExcelWriter('Fuzzy TOPSIS rankings.xlsx', engine='xlsxwriter')
b = 1
for a in range(0,len(Fuzzy_wts)):
    sens_wts_topsis = pd.read_excel("Sensitivity Weights for TOPSIS.xlsx", sheet_name ="sens_wts_TOPSIS" + str(b))
    sens_wts_topsis = sens_wts_topsis.set_index("Criteria")
    #print(sens_wts_topsis)
    b += 1

    Rating_Matrix = np.array(Rating_Matrix)
    Fuzzy_wts = np.array(sens_wts_topsis)
    Fuzzy_Eval_vals = np.dot(Rating_Matrix, sens_wts_topsis)
    Fuzzy_Eval_vals = pd.DataFrame(Fuzzy_Eval_vals)
    Fuzzy_Eval_vals.columns = ["l","m","n"]
    #print(Fuzzy_Eval_vals)


    row_index = ['A' + str(i) for i in range(1, len(Fuzzy_Eval_vals)+1)]
    Fuzzy_Eval_vals["Stretch No."] = row_index
    Fuzzy_Eval_vals_1 = Fuzzy_Eval_vals.set_index("Stretch No.")
    #print(Fuzzy_Eval_vals_1)

    
    Triangular_FuzzyNums = pd.DataFrame(columns = ['l','m','n'])
    for e in range(0,len(Fuzzy_Eval_vals)):
        for f in range(0,len(Fuzzy_Eval_vals)):
            arr1 = Fuzzy_Eval_vals.loc[e]
            arr2 = Fuzzy_Eval_vals.loc[f]
            diff = [arr1[0]-arr2[2],arr1[1]-arr2[1],arr1[2]-arr2[0]]
            stretch_diff = pd.Series(diff, index = Triangular_FuzzyNums.columns)
            Triangular_FuzzyNums = Triangular_FuzzyNums.append(stretch_diff, ignore_index=True)
    row_diff = ['p' + str(e) + "-" + "p" + str(f) for e in range(1, len(Fuzzy_Eval_vals)+1) for f in range(1, len(Fuzzy_Eval_vals)+1)]
    Triangular_FuzzyNums["p~i-p~j"] = row_diff
    Triangular_FuzzyNums_1 = Triangular_FuzzyNums.set_index("p~i-p~j")
    #print(Triangular_FuzzyNums_1)


    Intersection_pnts = []
    for k in range(0,len(Triangular_FuzzyNums)):
        l = Triangular_FuzzyNums.iloc[k,0]
        m = Triangular_FuzzyNums.iloc[k,1]
        n = Triangular_FuzzyNums.iloc[k,2]
    
        if l < 0 and m > 0:
            p1, p2, p3, p4 = Point(0, 0), Point(0, 1), Point(l, 0), Point(m, 1)
            l1 = Line(p1, p2)
            s1 = Segment(p3, p4)  
            showIntersection = l1.intersection(s1)
            #print(showIntersection)
            Intersection_pnts.append(showIntersection)
        elif m < 0 and n > 0: 
            p1, p2, p3, p4 = Point(0, 0), Point(0, 1), Point(n, 0), Point(m, 1)
            l1 = Line(p1, p2)
            s1 = Segment(p3, p4)  
            showIntersection = l1.intersection(s1)
            #print(showIntersection)
            Intersection_pnts.append(showIntersection)
        else:
            Intersection_pnts.append([[0,0]])

    
    Intersection = []
    for z in range(0,len(Triangular_FuzzyNums_1)):
        Intersection.append(Intersection_pnts[z][0][1])

    Triangular_FuzzyNums_1["Intersection height"] = Intersection
    #print(Triangular_FuzzyNums_1)

   
    E = []
    for d in range(0,len(Triangular_FuzzyNums)):
        l = Triangular_FuzzyNums_1.iloc[d,0]
        m = Triangular_FuzzyNums_1.iloc[d,1]
        n = Triangular_FuzzyNums_1.iloc[d,2]
        h = round(Triangular_FuzzyNums_1.iloc[d,3], 3)

        if l < 0 and m > 0 and n > 0:
            pos_Area = (1/2 * (n-m)) + (1/2 * (m) * (1-h)) + (h * m)
            neg_Area = 1/2 * (0-l) * h
        elif m < 0 and n > 0:
            pos_Area = 1/2 * (n) * h
            neg_Area = (1/2 * abs((m-l))) + (1/2 * (0-m) * (1-h)) + ((0-m) * h)
        elif n < 0:
            pos_Area = 0
            neg_Area = (1/2 * (n-m)) + (1/2 * (m-l))
        else:
            pos_Area = (1/2 * (n-m)) + (1/2 * (m-l))
            neg_Area = 0

        total_Area = pos_Area + neg_Area
        corr_e = round(pos_Area/total_Area, 3)
        E.append(corr_e)
        
    E_array = np.array(E)
    TriFuzzNo_sqrt = int(np.sqrt(len(Triangular_FuzzyNums)))
    E_2D_array = np.reshape(E_array, (TriFuzzNo_sqrt, TriFuzzNo_sqrt))
    np.fill_diagonal(E_2D_array, 0.5)
    #print(E_2D_array)

    
    PI = []
    for x in range(0,len(E_2D_array)):
        PI_Sum = sum(E_2D_array[x]) - (len(E_2D_array) * 0.5)
        PI.append(PI_Sum)
        
    Ranking = pd.DataFrame()
    Ranking["Priority Index"] = PI
    Ranking["Stretch No."] = row_index
    Ranking["Rank"] = Ranking["Priority Index"].rank(ascending = False, method = 'max')
    Ranking = Ranking.set_index("Stretch No.")
    
    #print(str(a+1), Ranking)
    Ranking.to_excel(Topsis, sheet_name = "topsis_rank" + str(a+1) + "_" +str(user_sensitivity_val))
Topsis.save()